# PLAsTiCC time series classification
This is a ML code that performs classification of the time series from the Photometric LSST Astronomical Time-Series Classification Challenge (PLAsTiCC; https://www.kaggle.com/c/PLAsTiCC-2018). A sample representation of the data is provided after loading the data.

Labels are provided for the training set.

The steps are the following: <br>
1. Import libraries and sample
2. Split sub-samples (training, validation, testing)
3. Fit with various classifier and check performance
4. Compare various classifiers in testing sample

The alternative classifiers provide first guesses on the expected accuracy

## HISTORY:

v3.0:
- Using the power spctrum approach in 1D

## Loading Data

In [1]:
# Importing Libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import datasets, utils, metrics
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.simplefilter('ignore')

In [2]:
path_to_table    = "/data/deep_learning/database_PLAsTiCC/training_set.csv"
path_to_metadata = "/data/deep_learning/database_PLAsTiCC/training_set_metadata.csv"

In [3]:
table = np.genfromtxt(path_to_table   , delimiter=',', names=True)
meta  = np.genfromtxt(path_to_metadata, delimiter=',', names=True)

print("Data")
print('Type: '  + str(type(table)))
print('Shape: ' + str(table.shape))
print('Keys: '  + str(table.dtype.names))

print()

print("Meta data")
print('Type: '  + str(type(meta)))
print('Shape: ' + str(meta.shape))
print('Keys: '  + str(meta.dtype.names))


Data
Type: <class 'numpy.ndarray'>
Shape: (1421705,)
Keys: ('object_id', 'mjd', 'passband', 'flux', 'flux_err', 'detected')

Meta data
Type: <class 'numpy.ndarray'>
Shape: (7848,)
Keys: ('object_id', 'ra', 'decl', 'gal_l', 'gal_b', 'ddf', 'hostgal_specz', 'hostgal_photoz', 'hostgal_photoz_err', 'distmod', 'mwebv', 'target')


In [4]:
classes = np.unique(meta['target'])
classes_str = ["Class "+str(int(cl)) for cl in classes]
n_classes = len(classes)

n_bands  = 6


idx_unique = np.where(np.unique(meta['object_id']))[0]
# indices of unique target/labels

IDs_unique = table['object_id'][idx_unique]
# list of target IDs, without repetitions

labels_unique = meta['target'][idx_unique]


print('Number of targets:               ' + str(len(IDs_unique)))        
print('Number of labels:                ' + str(len(labels_unique)))     
print()
print('Number of possible classes:      ' + str(n_classes))        
print('Number of pixels per periodgram: ' + str(n_pixels))
print('Number of bands:                 ' + str(n_bands))

Number of targets:               7848
Number of labels:                7848

Number of possible classes:      14
Number of pixels per periodgram: 100
Number of bands:                 6


## Converting lightcurves into periodgrams

In [ ]:
import progressbar

from astroML.time_series import lomb_scargle, lomb_scargle_BIC, lomb_scargle_bootstrap

periodgrams = np.zeros( (len(IDs_unique) , n_pixels , n_bands ) )
# data array < n_targets , image(x,y), bands >
# NOTE: The first dimension corresponds to the number of targets <indexed as IDs_unique>

n_pixels = 100
# numebr of elements in each periodgram


bar = progressbar.ProgressBar(maxval=len(IDs_unique), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
# restore:
for i in range(len(IDs_unique)):
# i = ID index

    bar.update(i+1)

    table_obj = table[table['object_id'] == IDs_unique[i]]

    for b in range(n_bands):

        table_obj_b = table_obj[table_obj['passband'] == b]
        
        # Series for given object and band:
        MJD   = table_obj_b['mjd']
        F     = table_obj_b['flux']
        F_err = table_obj_b['flux_err']
        
        # log scaling: period = 10 ** np.linspace(-1, 2, n_pixels)
        # linear scaling:
        period = np.linspace(1, 50, n_pixels)
        omega  = 2 * np.pi / period
        periodgram_obj_b = lomb_scargle(MJD, F, F_err, omega, generalized=True)

        # Normalizing periodgram:
        periodgram_obj_b_norm = periodgram_obj_b / np.max(periodgram_obj_b)
        
        periodgrams[i,:,b] = periodgram_obj_b
        
bar.finish()        

print('Shape of periodgrams: ' + str(periodgrams.shape))

In [ ]:
def plot_lightcurve(idx_c_i, xrange=None):
    
    fig, ax = plt.subplots(2, 3, figsize=(12, 6))
    
    # Iterating over bands:
    for i in range(3):
        for j in range(2):
            b = 2*i+j
            
            table_obj = table[table['object_id'] == IDs_unique[idx_c_i]]

            table_obj_b = table_obj[table_obj['passband'] == b]
        
            # Series for given object and band:
            MJD   = table_obj_b['mjd']
            F     = table_obj_b['flux']
            F_err = table_obj_b['flux_err']
                            
            ax[j,i].errorbar(MJD,
                             F,
                             yerr=F_err,
                             fmt='o', marker='.', ecolor='k', markersize=10)
            
            ax[j,i].set_title('Filter '+str(int(b)))
            
            if not xrange is None:
                ax[j,i].set_xlim(xrange[0], xrange[1])
                    
    plt.tight_layout()
    plt.show()

def plot_periodogram(idx_c_i):
    
    fig, ax = plt.subplots(2, 3, figsize=(12, 6))
    
    # Iterating over bands:
    for i in range(3):
        for j in range(2):
            b = 2*i+j

            periodgram_obj_b = periodgrams[idx_c_i,:,b]
            
            period = np.arange(len(periodgram_obj_b))

            ax[j,i].plot(period, periodgram_obj_b, '-', c='black', lw=1, zorder=1)
                        
            ax[j,i].set_title('Filter '+str(int(b)))

    plt.tight_layout()
    plt.show()

            
for c in range(n_classes):
# c = class index
    
    print("Class " + str(int(classes[c])))

    
    #idx = np.where(table_obj['passband'] == i_filter)[0]

    idx_c = np.where(meta['target'] == classes[c])[0]
    # indexes of targets of class = c
    
    idx_c_0 = idx_c[0]
    # index of first target of class = c
 
    plot_lightcurve(idx_c_0)
    plot_periodogram(idx_c_0)
        
plt.tight_layout()
plt.show()            

## Creating samples


In [ ]:
# Shuffle the samples
shuffled_indexes = np.arange(len(IDs_unique))
np.random.shuffle(shuffled_indexes)

# To reduce the sample size (for testing purposes):
# remove: shuffled_indexes = shuffled_indexes[0:1000]
# remove: n_samples = len(shuffled_indexes)

periodgrams_shuffled = periodgrams[shuffled_indexes]
labels_shuffled = list(np.array(labels_unique)[shuffled_indexes])

n_samples = len(IDs_unique)

In [ ]:
data   = periodgrams_shuffled
labels = labels_shuffled

# Splitting in training, validation, and test samples:
data_train = data[:8 * n_samples // 10] # i.e. 80% training
labels_train = labels[:8 * n_samples // 10]

data_valid = data[8 * n_samples // 10:9 * n_samples // 10] # i.e. 10% validation (80->90%)
labels_valid = labels[8 * n_samples // 10:9 * n_samples // 10]

data_test = data[9 * n_samples // 10:] # i.e. 10% testing (90->100%)
labels_test = labels[9 * n_samples // 10:]

#n_train_spiral = len([x for x in labels_train if x == 'spiral'])
#n_train_ellipt = len([x for x in labels_train if x == 'ellipt'])

#n_valid_spiral = len([x for x in labels_valid if x == 'spiral'])
#n_valid_ellipt = len([x for x in labels_valid if x == 'ellipt'])

#n_test_spiral = len([x for x in labels_test if x == 'spiral'])
#n_test_ellipt = len([x for x in labels_test if x == 'ellipt'])

print("Sample Summary")
print("________________________")
print("Total images     | %5s" % len(data))
print("-----------------|------")
print(" '-> Training    | %5s" % len(data_train))
#print("      '-> spiral | %5s (%.1f%%)" % (n_train_spiral , (n_train_spiral/len(data_train)*100.)))
#print("      '-> ellipt | %5s (%.1f%%)" % (n_train_ellipt , (n_train_ellipt/len(data_train)*100.)))
print("-----------------|------")
print(" '-> Validation  | %5s" % len(data_valid))
#print("      '-> spiral | %5s (%.1f%%)" % (n_valid_spiral , (n_valid_spiral/len(data_valid)*100.)))
#print("      '-> ellipt | %5s (%.1f%%)" % (n_valid_ellipt , (n_valid_ellipt/len(data_valid)*100.)))
print("-----------------|------")
print(" '-> Test        | %5s" % len(data_test))
#print("      '-> spiral | %5s (%.1f%%)" % (n_test_spiral , (n_test_spiral/len(data_test)*100.)))
#print("      '-> ellipt | %5s (%.1f%%)" % (n_test_ellipt , (n_test_ellipt/len(data_test)*100.)))

print('')
print('Compare these values with the accuracy of each classifier')
print('If accuracies are similar to the demographics, the classifier is only mirroring the data')

## Converting to 1 band for other classifiers


In [ ]:
# Converting to 1 band:

# Using only 1 band, and compressing dimensions into 1:
data_train_1D = data_train[:,:,5].reshape(len(data_train),n_pixels)
data_valid_1D = data_valid[:,:,5].reshape(len(data_valid),n_pixels)

labels_train_1D = labels_train
labels_valid_1D = labels_valid 

print('Shape of train 1D       | ' + str(data_train_1D.shape))
print('Shape of valid 1D       | ' + str(data_valid_1D.shape))

print('Shape of label train 1D | ' + str(len(labels_train_1D)))
print('Shape of label valid 1D | ' + str(len(labels_valid_1D)))

## > Using scikit-learn / SVM classifier
Classical Support Vector Machines classifier
NOTE: The non-linear kernel is extremely slow

In [ ]:
# Classifier
model_svc = LinearSVC()
#model_svc = NuSVC()
model_svc.fit(data_train_1D, labels_train_1D)

# Comparisong with prediction
predicted = model_svc.predict(data_valid_1D)

print("Classification report for %s:\n%s\n"
      % (model_svc, metrics.classification_report(labels_valid_1D, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(labels_valid_1D, predicted))

## > Using scikit learn / Random Forests
A scikit-learn bagging classifier.

In [ ]:
# Classifier
model_RF = RandomForestClassifier()
model_RF.fit(data_train_1D, labels_train_1D)

# Comparisong with prediction
predicted = model_RF.predict(data_valid_1D)

print("Classification report for %s:\n%s\n"
      % (model_RF, metrics.classification_report(labels_valid_1D, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(labels_valid_1D, predicted))

## > Building a Keras Neural Network classifier


In [ ]:
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution1D
from keras.layers import Convolution2D
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import MaxPool1D
from keras.layers import MaxPool2D
from keras.layers import Dropout

from tensorflow.contrib.layers import maxout

from keras.utils import np_utils
from keras import regularizers
from keras import optimizers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
n_bands = data_train.shape[-1]

# >> One hot encoding the class values to tranform the vector of class integers into a binary matrix:
int_enc = LabelEncoder()
labels_train_int = int_enc.fit_transform(labels_train)
labels_valid_int = int_enc.fit_transform(labels_valid)
labels_test_int  = int_enc.fit_transform(labels_test)

labels_train_int = np.expand_dims(labels_train_int, axis=1)
labels_valid_int = np.expand_dims(labels_valid_int, axis=1)
labels_test_int  = np.expand_dims(labels_test_int, axis=1)

# Replicating the classification for all the bands:
#labels_train_int = np.repeat(labels_train_int[:,np.newaxis], n_bands, 1)
#labels_valid_int = np.repeat(labels_valid_int[:,np.newaxis], n_bands, 1)
#labels_test_int  = np.repeat(labels_test_int[:,np.newaxis], n_bands, 1)

oh_enc = OneHotEncoder(sparse=False)
labels_train_ohe = oh_enc.fit_transform(labels_train_int)
labels_valid_ohe = oh_enc.fit_transform(labels_valid_int)
labels_train_ohe = oh_enc.fit_transform(labels_test_int)

# uniques, labels_valid = np.unique(labels_valid, return_inverse=True)
labels_train_cat = np_utils.to_categorical(labels_train_int)
labels_valid_cat = np_utils.to_categorical(labels_valid_int)
labels_test_cat  = np_utils.to_categorical(labels_test_int)

#n_classes = labels_valid_ohe.shape[1]
#n_classes = 15
# must hard-code this one

print("Labels formats for convolutional layers:")

print("Train      int label format (?, ?, n_samples, n_channels)         | ", labels_train_int.shape)
print("Validation int label format (?, ?, n_samples, n_channels)         | ", labels_valid_int.shape)
print("Test       int label format (?, ?, n_samples, n_channels)         | ", labels_test_int.shape)

In [ ]:
n_pixels = data_train.shape[1] 

# Formatting data for convolutional layer:
n_train_targets = data_train.shape[0]
n_valid_targets = data_valid.shape[0]
n_test_targets  = data_test.shape[0]

#labels_train_int_4D = np.expand_dims(labels_train_int   , axis=0)
#labels_train_int_4D = np.expand_dims(labels_train_int_4D, axis=0)
#labels_valid_int_4D = np.expand_dims(labels_valid_int   , axis=0)
#labels_valid_int_4D = np.expand_dims(labels_valid_int_4D, axis=0)
#labels_test_int_4D  = np.expand_dims(labels_test_int    , axis=0)
#labels_test_int_4D  = np.expand_dims(labels_test_int_4D , axis=0)


print("Data formats for convolutional layers:")

print("Train      4D data format (n_samples,size_x, size_y, n_channels) | ", data_train.shape)
print("Validation 4D data format (n_samples,size_x, size_y, n_channels) | ", data_valid.shape)
print("Test       4D data format (n_samples,size_x, size_y, n_channels) | ", data_test.shape)

#print("Train      4D label format (?, ?, n_samples, n_channels)         | ", labels_train_int_4D.shape)
#print("Validation 4D label format (?, ?, n_samples, n_channels)         | ", labels_valid_int_4D.shape)
#print("Test       4D label format (?, ?, n_samples, n_channels)         | ", labels_test_int_4D.shape)

In [ ]:
# Trying with less bands:

n_bands = 6

data_train = data_train[:,:,:n_bands]
data_valid = data_valid[:,:,:n_bands]
data_test  = data_test [:,:,:n_bands]

if (n_bands == 1):
    data_train = np.expand_dims(data_train, axis=3)
    data_valid = np.expand_dims(data_valid, axis=3)
    data_test  = np.expand_dims(data_test, axis=3)


print("Data formats for convolutional layers - 1 band:")

print("Train      4D data format (n_samples,size_x, size_y, n_channels) | ", data_train.shape)
print("Validation 4D data format (n_samples,size_x, size_y, n_channels) | ", data_valid.shape)
print("Test       4D data format (n_samples,size_x, size_y, n_channels) | ", data_test.shape)

### Convolutional 1D layers

In [ ]:
model_Conv = keras.Sequential([
                          #keras.layers.Conv1D(8, kernel_size=4, strides=2, padding='same',
                          #                    activation=tf.nn.relu, input_shape=(n_pixels,n_bands)),
                          #keras.layers.MaxPool1D(pool_size=2, strides=2),
                          #keras.layers.Conv1D(8, kernel_size=4, strides=2, padding='same',
                          #                    activation=tf.nn.relu, input_shape=(n_pixels,n_bands)),
                          #keras.layers.MaxPool1D(pool_size=2, strides=2),
                          #keras.layers.Dropout(0.3),
                          keras.layers.Dense(20, activation=tf.nn.sigmoid, input_shape=(n_pixels,n_bands)),
                          keras.layers.Dense(50, activation=tf.nn.sigmoid),
                          keras.layers.Dense(20, activation=tf.nn.sigmoid),
                          keras.layers.Dropout(0.3),
                          keras.layers.Flatten(),
                          keras.layers.Dense(n_classes, activation=tf.nn.sigmoid)
                          ])

model_Conv.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

model_Conv.summary()

In [ ]:
history_Conv = model_Conv.fit(data_train, labels_train_int, validation_data=(data_valid, labels_valid_int),
                    epochs=10, batch_size=250, verbose=2)

In [ ]:
# > Model evolution:
plt.plot(history_Conv.history['loss'])
plt.plot(history_Conv.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()


print('Number of different classes in train sample: ' + str(len(np.unique(labels_train))))
print('Number of different classes in valid sample: ' + str(len(np.unique(labels_valid))))
print('Number of different classes in test  sample: ' + str(len(np.unique(labels_test))))
print('')
print('WARNING: A disply error can occur if the represented classes differ in the train/valid/test sets.')
print('         If so, increase the samples.')


# > Comparison with predictions:
labels_pred_float_Conv = model_Conv.predict(data_test)

labels_pred_Conv = int_enc.inverse_transform(labels_pred_float_Conv.argmax(1))
# reversing one hot encoding


print("Classification report for %s:\n%s\n"
      % (model_Conv, metrics.classification_report(labels_test, labels_pred_Conv)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(labels_test, labels_pred_Conv))


# > Plotting a few misclassified images (wrong labels):

print('Examples of misclassified images:')

# equivalent(?): indexes_wrong = np.nonzero(labels_pred_Conv != labels_test_int)
indexes_wrong = [labels_pred_Conv != labels_test]
# indexes of misclassified objects <indexed within range(data_test) or range(labels_pred_*)>

labels_pred_int_Conv = labels_pred_float_Conv.argmax(1)

data_test_wrong            = data_test[indexes_wrong]
labels_test_int_wrong      = labels_test_int[indexes_wrong]
labels_pred_int_Conv_wrong = labels_pred_int_Conv[indexes_wrong]

fig = plt.figure(figsize=(15,10))
plt.subplots_adjust(hspace=0.25,wspace=0.05)

       
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.axis('off')
    peridogram_wrong = (data_test_wrong[i,:,0].reshape(n_pixels))
    # using only first band in array
    
    vmin,vmax = np.percentile(image_wrong.flatten(),[1,99])
    # min and max are set as percentiles for each image

    plt.plot(peridogram_wrong)
    plt.title('label = %s | pred = %s' % (labels_shuffled[labels_test_int_wrong[i][0]] , labels_shuffled[labels_pred_int_Conv_wrong[i]]))



https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/

## Using Tensorflow
https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/neural_network_raw.ipynb

https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/neural_network.ipynb

## Final check 
Using the test sample for a final check of the various algorithms used above. 

Checking if their performance is as good as it is reported in the validation process.

In [ ]:
print("====================================================================================")
# Predictions for SVM:
predicted = model_svc.predict(data_test_1D)
print("Classification report for %s:\n%s\n"
      % (model_svc, metrics.classification_report(labels_test, predicted)))
print("Confusion matrix:\n%s \n" % metrics.confusion_matrix(labels_test, predicted))
print("====================================================================================")

# Predictions for LogisticRegression:
#predicted = model_svc.predict(data_test)
#print("Classification report for %s:\n%s\n"
#      % (model_lrc, metrics.classification_report(labels_test, predicted)))
#print("Confusion matrix:\n%s \n" % metrics.confusion_matrix(labels_test, predicted))
#print("====================================================================================")

# Predictions for MLP:
#predicted = model_MLP.predict(data_test)
#print("Classification report for %s:\n%s\n"
#      % (model_MLP, metrics.classification_report(labels_test, predicted)))
#print("Confusion matrix:\n%s \n" % metrics.confusion_matrix(labels_test, predicted))
#print("====================================================================================")

# Predictions for RandomForests:
predicted = model_RF.predict(data_test_1D)
print("Classification report for %s:\n%s\n"
      % (model_RF, metrics.classification_report(labels_test, predicted)))
print("Confusion matrix:\n%s \n" % metrics.confusion_matrix(labels_test, predicted))
print("====================================================================================")

# Predictions for Keras 1D Neural Network:
#labels_pred_float_1D = model_1D.predict(data_test)
#labels_pred_1D = int_enc.inverse_transform(labels_pred_float_1D.argmax(1))
#print("Classification report for %s:\n%s\n"
#      % (model_1D, metrics.classification_report(labels_test, labels_pred_1D)))
#print("Confusion matrix:\n%s" % metrics.confusion_matrix(labels_test, labels_pred_1D))
#print("====================================================================================")

# Predictions for Keras Convolutional Neural Network:
labels_pred_float_Conv = model_Conv.predict(data_test)
labels_pred_Conv = int_enc.inverse_transform(labels_pred_float_Conv.argmax(1))
print("Classification report for %s:\n%s\n"
      % (model_Conv, metrics.classification_report(labels_test, labels_pred_Conv)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(labels_test, labels_pred_Conv))
print("====================================================================================")

# Saving the trained models
Using pickle we can save the trained models to use them in a later time w/o the need to re-train them.

In [ ]:
#import pickle
#import os

## saving the models to disk
#folder_saved = "saved_models/v3.0"

#model_labels = ['SVC','LogReg','MLP','RandFor','Ker_1D','Ker_Conv']
#models = ['model_SVC','model_LR','model_MLP','model_RF','model_1D','model_Conv']

#if not os.path.exists(folder_saved):
#    os.makedirs(folder_saved)
    
#for modstr,model in zip(model_labels,models):
#    pickle.dump(model, open(folder_saved+'/galaxy_classification_SDSS_v1_'+modstr+'.sav', 'wb'))


# To load the models at a later time:
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)